## Introduction
This sample notebook shows how to log metrics to TensorBoard. For those not familiar with TensorBoard, it is a visualization tool originally developed for TensorFlow but now also very usable for other frameworks like PyTorch.

In this example there are two different types of information logged:

1. A plain scalar metric (the validation loss) using the `TensorBoardMeter`
2. A histogram of the gradients and weights of the model using the `ParamHistogram`

Under the hood, both classes use `TensorBoardX` to write data to the files. This is done by passing 
a writer object as an argument to the above classes.

For the rest this notebook uses the same dummy data and `resnet18` model as the basic_fos example.
So check that one out if not all is clear.


In [1]:
import torch
import torch.nn.functional as F
from torchvision.models import resnet18 

from tensorboardX import SummaryWriter

# Import the  Fos classes we'll use
from fos import Supervisor, Trainer 
from fos.meters import NotebookMeter, TensorBoardMeter, MultiMeter, AvgCalc
from fos.metrics.modelmetrics import ParamHistogram

## Setup
We first create an instance of the model we want to train, so in this case the resnet18 model. Throughout this example we refer to the instance of model as the `predictor` in order to be able to differentiate from the supervisor we create later on.

After the predictor is instantiated, the optimizer and loss function are created. If you are familiar with PyTorch this should all be straight forward. We choose Adam as the optimizer since it performs well in many scenarios. But feel free to swap it out for any other optimizer. And as a loss function we choose the `binary cross entropy` as that fits the multi-class classification problem well.

And finally time to create some random dummy data that mimics an image of 224x224 pixels and the target: 

     X: 4x3x224x224 = 4 samples in a batch x 3 channels (RGB) x 224 pixels width x 224 pixels height
     
     Y: 4x1000      = 4 samples in a batch x 1000 possible classes 

In a real world scenario's this would typically be implemented as a PyTorch Dataloader. But for the purpose of this notebook a simple list of random tensors will do just fine.

In [2]:
predictor = resnet18()
optim     = torch.optim.Adam(predictor.parameters())
loss      = F.binary_cross_entropy_with_logits

data = [(torch.randn(4,3,224,224), torch.rand(4,1000).round()) for i in range(10)]

## Setup metrics
We create a writer to that will handle creating the nesecarry files and writing the data to these files. The SummaryWriter is part of the excellent TensorBoardX package.

After we have an instance of the writer, we can create a TensorBoardMeter. In this example it will only process and store the `val_loss` metric.

Finally we create the ParamHistogtam metric. This is a model metric, in other words checks the performance of the model and not the performance of the predictions. In this case it will create histograms of the weights and gradients of the layers in the model. We pass the same writer as an argument. You can use a different writer, but then it should also write to a different file.

In [3]:
writer  = SummaryWriter("./runs/my_run")
nbmeter = NotebookMeter()
tbmeter = TensorBoardMeter(writer)
meter   = MultiMeter(nbmeter, tbmeter)

model   = Supervisor(predictor, loss)
metric  = ParamHistogram(writer, skip=10)
trainer = Trainer(model, optim, meter, metrics=[metric])

## Train
Now we train the model. In order to see the tensorbaord output, you'll need to start tensorboard and point it to the directory where this norebook is logging its data. 

In [4]:
trainer.run(data, epochs=5)

[  0:     9] loss=0.73209 : 100%|██████████|00:05<00:00
[  1:    19] loss=0.66593 : 100%|██████████|00:05<00:00
[  2:    29] loss=0.63975 : 100%|██████████|00:05<00:00
[  3:    39] loss=0.60539 : 100%|██████████|00:05<00:00
[  4:    49] loss=0.56704 : 100%|██████████|00:05<00:00
